In [5]:
from src.api.server_client import ServerClientWeaveflowAPI

In [6]:
api = ServerClientWeaveflowAPI("http://127.0.0.1:6345")
def with_scope(data):
    return {
        **data,
        "entity": "test_entity",
        "project": "test_project"
    }

def with_other_scope(data):
    return {
        **data,
        "entity": "test_entity_other",
        "project": "test_project_other"
    }

In [8]:
import datetime
num_calls = 300
start_time = datetime.datetime.now()
for i in range(num_calls):
    create_res = api.call_create({"call": {"entity": "test_entity", "project": "test_project", 'op_name': "test_op"}})
end_time = datetime.datetime.now()
time_elapsed = end_time - start_time
print("DONE", time_elapsed, time_elapsed / (i + 1))
read_res = api.calls_query(with_scope({}))
print(len(read_res.calls))

DONE 0:00:02.564444 0:00:00.008548
600


In [18]:
read_res = api.calls_query(with_scope({"offset": 10, "limit": 10}))
read_res.calls[-1]

PartialCallSchema(entity='test_entity', project='test_project', id='95fd8b17-99f9-4708-9ab8-c61c021b5247', trace_id='ed7b64ca-c8f6-4034-bc5e-0998df786497', parent_id=None, op_name='test_op', status_code=<StatusCodeEnum.UNSET: 'UNSET'>, start_time=1708048397.0, end_time=None, attributes={'call_num': 80}, inputs=None, outputs=None, summary=None, exception=None)

In [9]:
create_res = api.call_create({"call": with_scope({'op_name': "Tims_op1", "attributes": {"test": 1}})})
create_res = api.call_create({"call": with_other_scope({'op_name': "Tims_op2"})})
create_res = api.call_create({"call": with_scope({'op_name': "Tims_op3"})})
create_res

CallCreateRes(entity='test_entity', project='test_project', id='b5eb9d59-e1af-4e22-952d-9f840cc3c273')

In [10]:
read_res = api.call_read(with_scope({"id":create_res.id, "columns": ["attributes"]}))
read_res

CallReadRes(call=PartialCallSchema(entity='test_entity', project='test_project', id='b5eb9d59-e1af-4e22-952d-9f840cc3c273', trace_id=None, parent_id=None, op_name=None, status_code=None, start_time=None, end_time=None, attributes=None, inputs=None, outputs=None, summary=None, exception=None))

In [11]:
import datetime
read_res = api.call_update(with_scope({
    "id": create_res.id, 
    "fields": {
        "exception":"AHH",
        "summary": {
            "some": "value"
        },
    }
}))
read_res

CallUpdateRes(entity='test_entity', project='test_project', id='b5eb9d59-e1af-4e22-952d-9f840cc3c273')

In [12]:
read_res = api.call_read(with_scope({"id":create_res.id}))
read_res

CallReadRes(call=PartialCallSchema(entity='test_entity', project='test_project', id='b5eb9d59-e1af-4e22-952d-9f840cc3c273', trace_id='3f33295a-3359-4635-a6dd-0e6ab98f3c72', parent_id=None, op_name='Tims_op3', status_code=<StatusCodeEnum.ERROR: 'ERROR'>, start_time=1736853.0, end_time=1708053272.0, attributes=None, inputs=None, outputs=None, summary={'some': 'value'}, exception='AHH'))

In [13]:
query_res = api.calls_query(with_scope({"filter": {"call_ids": None}, "columns":None, "order_by": [("end_time", "ASC")]}))
query_res.calls

[PartialCallSchema(entity='test_entity', project='test_project', id='b5eb9d59-e1af-4e22-952d-9f840cc3c273', trace_id='3f33295a-3359-4635-a6dd-0e6ab98f3c72', parent_id=None, op_name='Tims_op3', status_code=<StatusCodeEnum.ERROR: 'ERROR'>, start_time=1736853.0, end_time=1708053272.0, attributes=None, inputs=None, outputs=None, summary={'some': 'value'}, exception='AHH'),
 PartialCallSchema(entity='test_entity', project='test_project', id='43afbe57-411e-449f-a45e-44a79906e307', trace_id='99abba99-ebdd-4e87-a3ac-04abd32f9655', parent_id=None, op_name='Tims_op1', status_code=<StatusCodeEnum.UNSET: 'UNSET'>, start_time=1708053265.0, end_time=None, attributes={'test': 1}, inputs=None, outputs=None, summary=None, exception=None),
 PartialCallSchema(entity='test_entity', project='test_project', id='feec6b30-018a-4801-b26f-16513b32c716', trace_id='a1f556b6-0b93-4adc-9205-c0b3852d929d', parent_id=None, op_name='test_op', status_code=<StatusCodeEnum.UNSET: 'UNSET'>, start_time=1708053213.0, end_ti

In [10]:
query_res = api.calls_query(with_scope({"filter": {"op_names": [read_res.call.op_name]}}))
len(query_res.calls)

1

In [11]:
query_res = api.calls_query(with_scope({"filter": {"op_names": [read_res.call.op_name], "trace_ids": [read_res.call.trace_id]}}))
len(query_res.calls)

1

In [32]:
import clickhouse_connect
ch_client = clickhouse_connect.get_client(host="localhost")
ch_client.command("OPTIMIZE TABLE calls FINAL")

In [49]:
res = ch_client.query("SELECT * FROM calls")
res.result_rows

[('test_entity',
  'test_project',
  '17eee48c-546c-47ae-8e64-046dba2dc263',
  '5a1a1d7e-05d0-4031-95e9-1ab6bf6c0107',
  None,
  'Tims_op2',
  'UNSET',
  datetime.datetime(2024, 2, 15, 15, 59, 55, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=57600), 'PST')),
  None,
  None,
  None,
  None,
  None,
  None,
  datetime.datetime(2024, 2, 15, 15, 59, 55, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=57600), 'PST'))),
 ('test_entity',
  'test_project',
  '378f6d6c-a14e-451d-89fe-c9a6fbaff1b3',
  '375e4a45-c187-4b59-98e4-3b02f471a229',
  None,
  'Tims_op1',
  'UNSET',
  datetime.datetime(2024, 2, 15, 15, 59, 52, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=57600), 'PST')),
  None,
  None,
  None,
  None,
  None,
  None,
  datetime.datetime(2024, 2, 15, 15, 59, 52, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=57600), 'PST'))),
 ('test_entity',
  'test_project',
  '49875463-9f37-4a6f-b470-0ea4d40a78ca',
  '4f5bd012-cce1-4cb5-884a-39

In [34]:
res = ch_client.query("SELECT id FROM calls WHERE id = {id:String}", parameters={'id': '0fba18aa-8886-4c31-b628-82304a14ed61'})

[]

In [27]:
res.result_rows

[('0fba18aa-8886-4c31-b628-82304a14ed61',)]